In [1]:
import tensorflow as tf
from src.yolov4.yolov4 import YOLOv4 as YOLOv4Model

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
from src.base_class import BaseClass